# KarpovCourses. Продуктовая аналитика 
## Сегментация клиентов 


### Домашнее задание. RFM-анализ



**Скачайте датасет и проведите RFM анализ. В каждом подсегменте поделите пользователей на 4 класса (как на занятии). Отсчитывайте количество дней, прошедших с момента последней покупки, с максимальной даты покупки в датасете.**

- Какое максимальное кол-во покупок было совершено одним пользователем?
- Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)
- Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?
- Какая верхняя граница у количества покупок у пользователей с классом 2 в подсегменте R?
- Сколько пользователей попало в сегмент 111?
- Сколько пользователей попало в сегмент 311?
- В каком RFM-сегменте самое большое кол-во пользователей?
- В каком RFM-сегменте самое маленькое кол-во пользователей?
- Какое количество пользователей попало в самый малочисленный сегмент?

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Matplotlib forms basis for visualization in Python
import matplotlib.pyplot as plt

# We will use the Seaborn library
import seaborn as sns
sns.set()

# Graphics in SVG format are more sharp and legible
%config InlineBackend.figure_format = 'svg' 

# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['image.cmap'] = 'viridis'

In [2]:
path = 'https://stepik.org/media/attachments/lesson/413464/RFM_ht_data.csv'
orders = pd.read_csv(path, parse_dates=['InvoiceDate'], low_memory=False)
#загружаем датасет

In [3]:
orders.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [4]:
orders.shape[0]

332730

In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332730 entries, 0 to 332729
Data columns (total 4 columns):
InvoiceNo       332730 non-null object
CustomerCode    332730 non-null object
InvoiceDate     332730 non-null datetime64[ns]
Amount          332730 non-null float64
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 10.2+ MB


In [6]:
#Приводим данные в нужный формат
orders['InvoiceDate'] = pd.to_datetime(orders['InvoiceDate'])
orders['CustomerCode'] = orders['CustomerCode'].apply(str)
orders['InvoiceNo'] = orders['InvoiceNo'].apply(str)

In [7]:
orders['CustomerCode'].apply(type).value_counts()

<class 'str'>    332730
Name: CustomerCode, dtype: int64

In [8]:
orders['CustomerCode'].nunique() 

123733

In [9]:
orders['InvoiceDate'].describe()

count                  332730
unique                     30
top       2020-09-30 00:00:00
freq                    19781
first     2020-09-01 00:00:00
last      2020-09-30 00:00:00
Name: InvoiceDate, dtype: object

### RFM-анализ

In [10]:
last_date = orders['InvoiceDate'].max()

In [11]:
rfmTable = orders.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (last_date - x.max()).days, # Recency #Количество дней с последнего заказа
                                        'InvoiceNo': lambda x: len(x),      # Frequency #Количество заказов
                                        'Amount': lambda x: x.sum()}) # Monetary Value #Общая сумма по всем заказам



In [12]:
rfmTable.head()

,InvoiceDate,InvoiceNo,Amount
CustomerCode,,,
02213019,19,1,1609.20
02213042,22,3,9685.48
02213071,29,1,415.00
02213088,23,1,305.00
02213092,25,1,1412.88


In [13]:
int(rfmTable.reset_index()['CustomerCode'].iloc[0])

2213019

In [14]:
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)

In [15]:
rfmSegmentation = rfmTable

In [16]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [17]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [18]:
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4

def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1

In [19]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)


In [20]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443


In [21]:
rfmSegmentation.shape[0]

123733

In [22]:
rfmSegmentation['frequency'].max()

204

In [23]:
rfmSegmentation[rfmSegmentation['RFMClass'] == '111'].shape[0]

9705

In [24]:
rfmSegmentation[rfmSegmentation['RFMClass'] == '311'].shape[0]

1609

In [25]:
rfmSegmentation.groupby('RFMClass').size().sort_values(ascending = False)

RFMClass
444    10624
111     9705
443     6729
344     6593
211     5847
       ...  
424       63
214       60
114       60
314       33
414        2
Length: 64, dtype: int64